In [ ]:
!pip3 install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [21]:
import pandas as pd
import numpy as np

csv_path = "trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)

In [22]:
x_columns = list(filter(lambda column: not 'punch' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
print(x_columns)
print(y_columns)

['trades-BUY-6-sec', 'trades-SELL-6-sec', 'trades-BUY-12-sec', 'trades-SELL-12-sec', 'trades-BUY-18-sec', 'trades-SELL-18-sec', 'trades-BUY-24-sec', 'trades-SELL-24-sec', 'trades-BUY-30-sec', 'trades-SELL-30-sec', 'trades-BUY-36-sec', 'trades-SELL-36-sec', 'trades-BUY-42-sec', 'trades-SELL-42-sec', 'trades-BUY-48-sec', 'trades-SELL-48-sec', 'trades-BUY-54-sec', 'trades-SELL-54-sec', 'trades-BUY-60-sec', 'trades-SELL-60-sec']
['punch-BUY-6-sec', 'punch-SELL-6-sec', 'punch-BUY-12-sec', 'punch-SELL-12-sec', 'punch-BUY-18-sec', 'punch-SELL-18-sec', 'punch-BUY-24-sec', 'punch-SELL-24-sec', 'punch-BUY-30-sec', 'punch-SELL-30-sec', 'punch-BUY-36-sec', 'punch-SELL-36-sec', 'punch-BUY-42-sec', 'punch-SELL-42-sec', 'punch-BUY-48-sec', 'punch-SELL-48-sec', 'punch-BUY-54-sec', 'punch-SELL-54-sec', 'punch-BUY-60-sec', 'punch-SELL-60-sec']


In [23]:
X = df[x_columns]
secs = list(reversed(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns]))))))

for side in ['BUY', 'SELL']:
  for s in secs[:-1]:
    X[f'trades-{side}-{str(s)}-sec'] -= X[f'trades-{side}-{str(s - (secs[0] - secs[1]))}-sec']
X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,trades-BUY-6-sec,trades-SELL-6-sec,trades-BUY-12-sec,trades-SELL-12-sec,trades-BUY-18-sec,trades-SELL-18-sec,trades-BUY-24-sec,trades-SELL-24-sec,trades-BUY-30-sec,trades-SELL-30-sec,trades-BUY-36-sec,trades-SELL-36-sec,trades-BUY-42-sec,trades-SELL-42-sec,trades-BUY-48-sec,trades-SELL-48-sec,trades-BUY-54-sec,trades-SELL-54-sec,trades-BUY-60-sec,trades-SELL-60-sec
0,0.100,0.200,0.400,0.300,10.255,0.200,0.100,0.500,0.300,0.200,0.216,2.700,0.200,0.200,0.200,2.066,0.000,2.600,0.500,9.981
1,0.200,0.400,0.300,3.684,0.200,0.366,0.300,0.400,0.250,0.000,0.200,0.200,0.749,0.400,0.100,0.000,0.200,33.531,0.800,0.100
2,0.200,2.434,49.298,139.475,0.300,4.486,0.100,5.982,0.200,5.806,0.400,7.355,0.246,9.057,0.100,0.300,0.220,1.000,0.200,0.300
3,2.513,9.616,4.975,5.386,36.317,35.898,0.200,8.477,0.000,0.300,0.400,5.400,9.516,0.150,0.000,0.200,0.000,2.181,0.200,24.480
4,5.152,0.200,0.100,6.006,0.100,9.416,6.790,0.400,9.316,0.100,1.167,0.100,0.500,9.616,2.439,0.200,161.648,0.200,6.105,9.969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,5.369,10.249,0.000,0.000,9.999,10.499,0.000,0.000,309.999,9.999,0.000,3.000,9.999,9.215,1.038,0.000,9.215,2.645,0.000,0.000
39942,7.765,20.339,0.000,0.414,9.999,9.999,0.000,0.000,0.000,26.878,1.914,0.000,0.000,25.819,0.035,0.000,0.000,101.502,0.000,5.583
39943,9.999,1.530,31.055,0.000,9.999,9.999,9.999,0.067,0.000,30.003,0.000,584.297,0.000,25.896,0.000,0.000,2.938,17.938,0.000,32.400
39944,0.802,1.381,0.861,0.000,5.582,0.000,10.000,3.005,10.000,86.202,0.000,0.000,90.906,155.992,19.788,48.927,1.134,0.000,197.118,0.000


In [24]:
commision = 0.0002
punch = commision * 4

y = pd.DataFrame(
    df[y_columns].apply(
        # lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        lambda row : 1 if max(abs(row.to_numpy())) > punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')
print(y)
print(y.value_counts())

       target
0           0
1           1
2           1
3           1
4           1
...       ...
39941       0
39942       0
39943       0
39944       1
39945       0

[39946 rows x 1 columns]
target
0         25936
1         14010
dtype: int64


In [25]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)

In [31]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    # 'depth': 5,
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'loss_function': 'Logloss',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
)
print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.7043


In [ ]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.MultiClass()
})
cv_data = cv(
    Pool(X, y),
    cv_params,
    plot=True
)
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [33]:
predictions = model.predict(X_test)
predictions = predictions.reshape(predictions.shape[0], 1)
predictions_probs = model.predict_proba(X_test)
print(predictions[10:20])
print(predictions_probs[10:20])
print(y_test[10:20])

[[0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]]
[[0.67611609 0.32388391]
 [0.86619178 0.13380822]
 [0.65935329 0.34064671]
 [0.63275776 0.36724224]
 [0.19702689 0.80297311]
 [0.25007291 0.74992709]
 [0.80752621 0.19247379]
 [0.61443975 0.38556025]
 [0.74729545 0.25270455]
 [0.82262985 0.17737015]]
       target
902         0
3804        0
33418       0
35239       0
18256       1
26745       1
7390        0
35376       0
23093       0
16538       0


In [34]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))


trades-BUY-6-sec: 7.470607652253464
trades-SELL-6-sec: 7.253465447980127
trades-BUY-18-sec: 6.101847265930916
trades-SELL-18-sec: 5.866337351323783
trades-SELL-24-sec: 5.2820557754218385
trades-SELL-54-sec: 5.129648003268254
trades-SELL-48-sec: 5.089088189223472
trades-BUY-30-sec: 5.01854024351934
trades-SELL-12-sec: 4.980854166302269
trades-SELL-30-sec: 4.966673396291865
trades-BUY-12-sec: 4.931175229515814
trades-BUY-24-sec: 4.730257634183482
trades-BUY-48-sec: 4.50149499002392
trades-BUY-60-sec: 4.471201511192456
trades-SELL-36-sec: 4.3770673465956955
trades-SELL-60-sec: 4.359567778160714
trades-SELL-42-sec: 4.169613690494425
trades-BUY-36-sec: 3.9735835628164073
trades-BUY-54-sec: 3.7297042566523424
trades-BUY-42-sec: 3.5972165088494434
